In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re,os, glob, traceback, nltk
from collections import defaultdict

In [4]:
my_personality_file = 'data/mypersonality.csv'
main_df = pd.read_csv(my_personality_file, encoding='Windows-1252')
main_df.shape

(9917, 20)

In [5]:
liwc_file = 'data/LIWC_mypersonality_oct_2.csv'
liwc_df = pd.read_csv(liwc_file)
liwc_df = liwc_df.drop(['Unnamed: 0', 'ColumnID', 'Text'], axis=1)
# liwc_df.fillna(value=np.nan, inplace=True)
# numerical_columns = liwc_df.select_dtypes(include=[np.number]).columns
# liwc_df[numerical_columns] = liwc_df[numerical_columns].fillna(liwc_df[numerical_columns].mean())
# liwc_df[numerical_columns] = liwc_df[numerical_columns].fillna(0)
# liwc_df.fillna(value=0, inplace=True)
liwc_df.head(1)


,#AUTHID,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,b7b7764cfa1c523e4e93ab2a79a946c4,1,5,99.0,NaN,NaN,99.0,5.0,20.0,80.0,...,0.0,0.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# df = pd.merge(main_df, liwc_df, on="#AUTHID", how="left")
df = pd.concat([main_df, liwc_df], axis=1)
df = df.drop(['#AUTHID', 'DATE'], axis=1)
str(list(df.columns))
# df.dropna(inplace=True)
df.shape

(9917, 137)

In [7]:
nrc_vad = pd.read_csv('data/NRC-VAD-Lexicon/NRC-VAD-Lexicon.csv', sep="\t")  
nrc_vad_dict = nrc_vad.set_index('Word').to_dict(orient='index')
def get_vad_scores(text):
    words = text.split()
    valence_scores, arousal_scores, dominance_scores = [], [], []
    for word in words:
        word = word.lower()  # Lowercase to match the lexicon
        if word in nrc_vad_dict:
            vad_values = nrc_vad_dict[word]
            valence_scores.append(vad_values['Valence'])
            arousal_scores.append(vad_values['Arousal'])
            dominance_scores.append(vad_values['Dominance'])
    if not valence_scores:
        return {'Valence': 0, 'Arousal': 0, 'Dominance': 0}

    valence_avg = sum(valence_scores) / len(valence_scores)
    arousal_avg = sum(arousal_scores) / len(arousal_scores)
    dominance_avg = sum(dominance_scores) / len(dominance_scores)
    return {'Valence': valence_avg, 'Arousal': arousal_avg, 'Dominance': dominance_avg}

df['VAD_Scores'] = df['STATUS'].apply( lambda x: get_vad_scores(x))
df[['Valence', 'Arousal', 'Dominance']] = pd.DataFrame(df['VAD_Scores'].tolist(), index=df.index)
df.drop(columns=['VAD_Scores'], inplace=True)
df.head(1)

,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,...,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji,Valence,Arousal,Dominance
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.765,0.623,0.398


In [8]:
nrc_lexicon = pd.read_csv('data/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', names=["word", "emotion", "association"],sep="\t", header=None)
# Filter out words that have no association with emotions (association == 0)
nrc_lexicon = nrc_lexicon[nrc_lexicon['association'] == 1]
# nrc_lexicon.drop(columns=['association'], inplace=True)
nrc_pivot = nrc_lexicon.pivot(index="word", columns="emotion", values="association").fillna(0).astype(int)
# nrc_pivot.head(2)
nltk.download('punkt')
def get_emotion_counts(text, lexicon):
    # print(text)
    words = nltk.word_tokenize(text.lower())
    emotion_count = defaultdict(int)
    for word in words:
        if word in lexicon.index:
            for emotion in lexicon.columns:
                emotion_count[emotion] += lexicon.loc[word, emotion]
    return emotion_count
emotion_counts_list = df['STATUS'].apply(lambda x: get_emotion_counts(x, nrc_pivot))
emotion_counts_df = pd.DataFrame(emotion_counts_list.tolist())
emotion_counts_df.fillna(0, inplace=True)
emotion_counts_df = emotion_counts_df.astype(int)
df = pd.concat([df, emotion_counts_df], axis=1)
df.head(1)

[nltk_data] Downloading package punkt to /home/jmaharja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,...,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def find_sentiment(text):
    # print(text)
    vs = analyzer.polarity_scores(text)
    sc = vs['compound']
    # emo = 'pos' if sc >= 0.05 else 'neu' if -0.05 < sc < 0.05 else 'neg'
    return sc
df[['sent_score']] = df['STATUS'].apply(lambda x: pd.Series(find_sentiment(x)))
df.head(1)

,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,...,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,sent_score
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,...,0,0,0,0,0,0,0,0,0,0.4215


In [10]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Step 1: Initialize the TfidfVectorizer
# You can specify parameters like max_features, ngram_range, etc., based on your needs
tfidf = TfidfVectorizer(max_features=100, stop_words='english')  # Adjust max_features as necessary

# Step 2: Fit and transform the 'STATUS' column
# This step converts the text in 'STATUS' to TF-IDF features
tfidf_matrix = tfidf.fit_transform(df['STATUS'].astype(str))  # Ensure 'STATUS' column is in string format

# Step 3: Convert the TF-IDF matrix into a DataFrame
# The resulting matrix is sparse, so we'll convert it to a DataFrame with feature names
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Step 4: Optionally, merge the TF-IDF features back with your original DataFrame
# This will add the new TF-IDF feature columns to your existing DataFrame
df = pd.concat([df, tfidf_df], axis=1)
df.head(1)

,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,...,watching,way,week,weekend,wishes,work,working,world,yay,year
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_embeddings(df, model_name, batch_size=8):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.to(device)
    model.eval()  
    embeddings_list = []
    
    for i in range(0, len(df), batch_size):
        batch_texts = df['STATUS'][i:i + batch_size].tolist()
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings_list.append(cls_embeddings.cpu().numpy())
    return np.vstack(embeddings_list)
    
bert_embeddings = get_embeddings(df, 'bert-base-uncased', batch_size=2)
roberta_embeddings = get_embeddings(df, 'roberta-base', batch_size=2)
berttweet_embeddings = get_embeddings(df, 'vinai/bertweet-base', batch_size=2)
xlnet_embeddings = get_embeddings(df, 'xlnet-base-cased', batch_size=2)


/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
df['bert_embeddings'] = list(bert_embeddings)
df['roberta_embeddings'] = list(roberta_embeddings)
df['berttweet_embeddings'] = list(berttweet_embeddings)
df['xlnet_embeddings'] = list(xlnet_embeddings)
# df[['STATUS', 'bert_embeddings', 'roberta_embeddings', 'berttweet_embeddings', 'xlnet_embeddings']].head(2)

In [20]:
dff = df.copy()
columns_to_change = ['cOPN', 'cEXT', 'cNEU', 'cAGR', 'cCON']
dff[columns_to_change] = dff[columns_to_change].replace({'y': 1, 'n': 0})
# dff.fillna(value=np.nan, inplace=True)
# numerical_columns = dff.select_dtypes(include=[np.number]).columns
# dff[numerical_columns] = dff[numerical_columns].fillna(dff[numerical_columns].mean())
dff.fillna(value=0, inplace=True)
print(dff.shape)
dff.head(1)

(9917, 255)


,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,...,wishes,work,working,world,yay,year,bert_embeddings,roberta_embeddings,berttweet_embeddings,xlnet_embeddings
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.5658656, -0.034683075, -0.3991114, -0.2992...","[-0.11116652, 0.09479402, -0.031377178, -0.120...","[-0.21966171, 0.3103059, 0.10933973, -0.048810...","[-0.5085498, 1.0680441, -0.029119104, -0.40934..."


In [22]:
all_cols = dff.columns
# liwc_df_cols = list(set(liwc_df.columns) - set(['#AUTHID', 'Text']))
label_cols = ['cEXT','cNEU', 'cAGR', 'cCON', 'cOPN']
remove_cols = ['#AUTHID', 'STATUS', 'sEXT', 'sNEU', 'sAGR', 'sCON', 'sOPN', 'cEXT','cNEU', 'cAGR', 'cCON', 'cOPN', 'DATE']
emb_cols = ['bert_embeddings', 'berttweet_embeddings', 'xlnet_embeddings', 'roberta_embeddings']
stat_cols = list (set(all_cols) - set(remove_cols) - set(emb_cols))
str(stat_cols), len(stat_cols)

("['swear', 'needs', 'lol', 'having', 'reward', 'good', 'got', 'adverb', 'power', 'god', 'people', 'propname', 'Analytic', 'politic', 'space', 'does', 'allure', 'night', 'wishes', 'yay', 'life', 'emo_anger', 'house', 'motion', 'maybe', 'comm', 'ppron', 'affiliation', 'mental', 'hours', 'trust', 'long', 'auxverb', 'acquire', 'make', 'Lifestyle', 'fulfill', 'certitude', 'gonna', 'differ', 'looking', 'class', 'verb', 'shehe', 'relig', 'let', 'Social', 'finally', 'family', 'they', 'money', 'facebook', 'better', 'work', 'i', 'number', 'emo_anx', 'Physical', 'emo_pos', 'know', 'wants', 'function', 'Comma', 'risk', 'Dominance', 'hope', 'thinks', 'loves', 'article', 'netspeak', 'tone_neg', 'sent_score', 'going', 'Period', 'WC', 'QMark', 'Emoji', 'substances', 'quantity', 'allnone', 'Culture', 'emo_sad', 'Perception', 'positive', 'wait', 'tone_pos', 'Cognition', 'friday', 'male', 'ipron', 'health', 'BigWords', 'thank', 'you', 'friends', 'getting', 'car', 'little', 'female', 'anger', 'AllPunc', 

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

stat_features = dff[stat_cols]
contextual_embeddings = np.array(dff["bert_embeddings"].tolist())

scaler = StandardScaler()  # This normalizes each feature to have mean=0 and std=1
stat_features_scaled = scaler.fit_transform(stat_features)
X = np.concatenate([stat_features_scaled, contextual_embeddings], axis=1)
y = np.array(dff[label_cols]) 

# Split data into train+val and test sets (80% train+val, 20% test)
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)
print(f'Train set size: {X_train.shape}')
print(f'Validation set size: {X_val.shape}')
print(f'Test set size: {X_test.shape}')

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
print(torch.isnan(X_train_tensor).any())  # Check for NaNs in training input
print(torch.isinf(X_train_tensor).any())   # Check for Infs in training input
print(torch.isnan(X_val_tensor).any())     # Check for NaNs in validation input
print(torch.isinf(X_val_tensor).any())      # Check for Infs in validation input
# shape : 240 + 768 = 1008

Train set size: (7933, 1008)
Validation set size: (992, 1008)
Test set size: (992, 1008)
tensor(False)
tensor(False)
tensor(False)
tensor(False)


In [42]:
stat_features

,swear,needs,lol,having,reward,good,got,adverb,power,god,...,emo_neg,right,week,fear,watching,WPS,la,doing,Authentic,ethnicity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,5.00,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.38,0.0,0.0,...,0.00,0.0,0.0,0,0.0,13.00,0.0,0.0,1.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,7.69,0.0,0.0,1,0.0,8.67,0.0,0.0,2.93,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.11,0.0,0.0,...,0.00,0.0,0.0,0,0.0,9.00,0.0,0.0,99.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,1.50,0.0,0.0,99.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,5.00,0.0,0.0,63.35,0.0
9913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,5.00,0.0,0.0,1.00,0.0
9914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,8.00,0.0,0.0,0.00,0.0
9915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0,0.0,4.00,0.0,0.0,0.00,0.0


In [36]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)  # Dropout to prevent overfitting
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [37]:
import torch.optim as optim
from sklearn.metrics import accuracy_score
model = MLP(input_size=X_train.shape[1], hidden_size=128, output_size=5)
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20
max_grad_norm=1.0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  
        outputs = model(inputs)
        if torch.isnan(outputs).any():
            print("NaN detected in model outputs!")
            break
        loss = criterion(outputs, labels)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        loss.backward()
        optimizer.step()
        # print(loss.item)
        total_loss += loss.item()
    # Validation phase
    model.eval()  
    val_preds = []
    val_labels = []
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()  # Calculate validation loss
            val_preds.append(torch.sigmoid(outputs))  # Collect predictions
            val_labels.append(labels)  # Collect true labels
    # Concatenate predictions and labels for accuracy calculation
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_preds = (val_preds > 0.5).float()  # Threshold to get binary predictions
    val_accuracy = accuracy_score(val_labels.numpy(), val_preds.numpy())
    # Print epoch results
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {total_loss / len(train_loader):.4f}, '
          f'Validation Loss: {val_loss / len(val_loader):.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')

Epoch [1/20], Train Loss: 0.6635, Validation Loss: 0.6556, Validation Accuracy: 0.0726
Epoch [2/20], Train Loss: 0.6431, Validation Loss: 0.6436, Validation Accuracy: 0.1099
Epoch [3/20], Train Loss: 0.6220, Validation Loss: 0.6319, Validation Accuracy: 0.1079
Epoch [4/20], Train Loss: 0.6024, Validation Loss: 0.6350, Validation Accuracy: 0.1472
Epoch [5/20], Train Loss: 0.5743, Validation Loss: 0.6521, Validation Accuracy: 0.1411
Epoch [6/20], Train Loss: 0.5463, Validation Loss: 0.6468, Validation Accuracy: 0.1472
Epoch [7/20], Train Loss: 0.5180, Validation Loss: 0.6645, Validation Accuracy: 0.1623
Epoch [8/20], Train Loss: 0.4873, Validation Loss: 0.6942, Validation Accuracy: 0.1734
Epoch [9/20], Train Loss: 0.4522, Validation Loss: 0.7129, Validation Accuracy: 0.1754
Epoch [10/20], Train Loss: 0.4294, Validation Loss: 0.7388, Validation Accuracy: 0.1653
Epoch [11/20], Train Loss: 0.3981, Validation Loss: 0.7639, Validation Accuracy: 0.1744
Epoch [12/20], Train Loss: 0.3800, Valida

In [68]:
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():  
    outputs = model(X_test_tensor)
outputs = torch.sigmoid(outputs)
preds = (outputs > 0.5).float()
preds_np = preds.numpy()
y_test_np = y_test  # If y_test is already in numpy format, otherwise y_test.numpy()
accuracy = accuracy_score(y_test_np, preds_np)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.2440


In [69]:
import torch.optim as optim
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.Wa = nn.Linear(hidden_dim * 2, hidden_dim * 2)  # For bidirectional
        self.Ua = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.Va = nn.Linear(hidden_dim * 2, 1)

    def forward(self, lstm_output):
        # lstm_output: (batch_size, seq_len, hidden_dim * 2)
        score = self.Va(torch.tanh(self.Wa(lstm_output)))  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(score, dim=1)  # Shape: (batch_size, seq_len, 1)
        context_vector = torch.bmm(attention_weights.permute(0, 2, 1), lstm_output)  # Shape: (batch_size, 1, hidden_dim * 2)
        return context_vector, attention_weights

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, bidirectional=True, dropout=0.5):
        super(BiLSTMClassifier, self).__init__()
        self.attention = Attention(hidden_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

    def forward(self, x):
        # if len(x.size()) == 2:
        #     x = x.unsqueeze(1)  # Add batch dimension to make it (1, seq_len, input_dim)
        # h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_dim).to(x.device)
        # c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_dim).to(x.device)
        lstm_out, _ = self.lstm(x)     
        context_vector, attention_weights = self.attention(lstm_out)
        context_vector = context_vector.squeeze(1)  # Shape: (batch_size, hidden_dim * 2)
        # out = lstm_out[:, -1, :]# Take the last time-step output
        out = self.fc(context_vector)
        return out, attention_weights

In [72]:
# Initialize model
model = BiLSTMClassifier(input_dim=768, hidden_dim=128, output_dim=5, num_layers=2)
input_data = torch.randn(32, 50, 768)  # Example input (batch_size=32, seq_len=50, input_dim=768)

# Forward pass
output, _ = model(input_data)
print(output.shape)  # Expected output: (batch_size, output_dim)

torch.Size([32, 5])


In [74]:
import torch.optim as optim
from sklearn.metrics import accuracy_score
model = BiLSTMClassifier(input_dim=X_train.shape[1], hidden_dim=128, output_dim=5,num_layers=2)
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20
max_grad_norm=1.0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for t, labels in train_loader:
        optimizer.zero_grad()  
        outputs, _ = model(t)
        # print(outputs.shape)
        # print(labels.shape)
        # print(t.shape)
        loss = criterion(outputs, labels)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # Validation phase
    model.eval()  
    val_preds = []
    val_labels = []
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()  # Calculate validation loss
            val_preds.append(torch.sigmoid(outputs))  # Collect predictions
            val_labels.append(labels)  # Collect true labels
    # Concatenate predictions and labels for accuracy calculation
    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_preds = (val_preds > 0.5).float()  # Threshold to get binary predictions
    val_accuracy = accuracy_score(val_labels.numpy(), val_preds.numpy())
    # Print epoch results
    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {total_loss / len(train_loader):.4f}, '
          f'Validation Loss: {val_loss / len(val_loader):.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')